In [36]:
# Standard library
import time
import os
import datetime
from concurrent.futures import ThreadPoolExecutor
import json
from web3 import Web3

# Third-party libraries
import pandas as pd
import requests
import psycopg2
from dotenv import load_dotenv

import duckdb

start_time = time.time()

load_dotenv()
subscan_key = os.getenv("subscan_key")
motherDuck_token = os.getenv("motherDuck_token")
onfinality_key = os.getenv("onfinality_key")
subscan_key = os.getenv("subscan_key")
MAX_WORKERS = 3  # adjust this based on your system's capabilities

# with duckdb.connect(f'md:origintrail?motherduck_token={motherDuck_token}&saas_mode=true') as conn:
#     try:
#
#         pass
#         # conn.execute(upsert_statement)
#         # print(f"Inserted {len(postgres_data)} rows.")
#     except Exception as e:
#         pass
#         # print(f"Error upserting data: {e}")


# Connect to the Ethereum node using Websockets
w3 = Web3(Web3.HTTPProvider(f'https://origintrail.api.onfinality.io/rpc?apikey={onfinality_key}'))

print("Latest Block:", w3.eth.block_number)
latest_block = w3.eth.block_number
last_block = (w3.eth.block_number) - 1

print(last_block, latest_block)
# Load ABI from json file
with open('data/ServiceAgreementV1.json', 'r') as file:
    serviceAgreementABI = json.load(file)

# Contract address and initialization
contract_address = '0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F'
contract = w3.eth.contract(address=contract_address, abi=serviceAgreementABI)


# Fetch past ServiceAgreementV1Created events
events_list = contract.events.ServiceAgreementV1Created.get_logs(fromBlock=last_block, toBlock=last_block)

dicts_list = [dict(event) for event in events_list]

# Flatten the 'args' dictionary and process HexBytes objects
processed_data = []
for item in dicts_list:
    flattened = {**item['args'], **item}  # Merge dictionaries
    flattened.pop('args')  # Remove the 'args' key

    # Convert HexBytes to hexadecimal string representation
    flattened['blockHash'] = flattened['blockHash'].hex()
    flattened['transactionHash'] = flattened['transactionHash'].hex()

    processed_data.append(flattened)

# Create DataFrame
df_assets = (pd.DataFrame(processed_data)
      .assign(tokenAmount=lambda x: x['tokenAmount'].astype(float) / 1e18,
              epochLength=lambda x: x['epochLength'].astype(float) / 86400,
              startTime=lambda x: pd.to_datetime(x['startTime'].apply(lambda y: datetime.datetime.utcfromtimestamp(y).isoformat())))
      .drop(columns=['keyword', 'hashFunctionId', 'logIndex', 'transactionIndex'])
      .rename(columns={"assetContract":"asset_contract", "startTime": "TIME_ASSET_CREATED", "epochsNumber":"EPOCHS_NUMBER","epochLength":"EPOCH_LENGTH-(DAYS)","tokenAmount": "TRAC_PRICE", "event":"EVENT","tokenId": "ASSET_ID", "assetContract":"ASSET_CONTRACT", "transactionHash":"TRANSACTION_HASH", "blockHash":"BLOCK_HASH", "blockNumber":"BLOCK_NUMBER", "address":"EVENT_CONTRACT_ADDRESS"}, errors="raise"))



# Get all transaction hashes
hashes = df_assets['TRANSACTION_HASH'].tolist()

hash_list = []

# Get all transaction hashes
for hash in hashes:
    url = "https://origintrail.api.subscan.io/api/scan/evm/transaction"
    headers = {
        "Content-Type": "application/json",
        "X-API-Key": subscan_key
    }
    data = {
        "hash": hash
    }

    response = requests.post(url, headers=headers, json=data).json()

    if response.get("code") == 0:  # only proceed if the response code indicates success
        data = response["data"]
        row = {
            "message": response["message"],
            "generated_at": response["generated_at"],
            "hash": data["hash"],
            "from": data["from"],
            "to": data["to"]["address"]
        }
        hash_list.append(row)

df_hash = ((pd.DataFrame(hash_list)
           .assign(generated_at=lambda x: pd.to_datetime(x['generated_at'].apply(lambda y: datetime.datetime.utcfromtimestamp(y).isoformat()))))
           .rename(columns={"message":"MESSAGE","generated_at":"TIME_OF_TRANSACTION","hash":"TRANSACTION_HASH","from":"PUBLISHER_ADDRESS", "to":"SENT_ADDRESS"}, errors="raise"))

df = pd.merge(df_assets, df_hash, on='TRANSACTION_HASH', how='left')

df = df[['MESSAGE', 'ASSET_ID', 'BLOCK_NUMBER', 'TIME_ASSET_CREATED', 'TIME_OF_TRANSACTION', 'TRAC_PRICE', 'EPOCHS_NUMBER', 'EPOCH_LENGTH-(DAYS)','PUBLISHER_ADDRESS', 'SENT_ADDRESS', 'TRANSACTION_HASH', 'BLOCK_HASH']]



end_time = time.time()
elapsed_time = end_time - start_time
print(f"Total execution time: {elapsed_time:.2f} seconds")





Latest Block: 3120320
3120319 3120320
Total execution time: 2.28 seconds


In [37]:
df

,MESSAGE,ASSET_ID,BLOCK_NUMBER,TIME_ASSET_CREATED,TIME_OF_TRANSACTION,TRAC_PRICE,EPOCHS_NUMBER,EPOCH_LENGTH,PUBLISHER_ADDRESS,SENT_ADDRESS,TRANSACTION_HASH,BLOCK_HASH
0,Success,561261,3120319,2023-08-25 21:20:18,2023-08-25 21:20:45,0.02124,5,7776000,0xa0a6ff7ee2d71d04c20195b8a919fafff513a9d4,0xb20016d23c77612334bd2885636980937da60b59,0x9ad586a86c548cff1de35f6e9daef09f2b44ff3a31bb...,0x86b72a67f9532e3f0ab7ebb691b26b2c3a6b31a1480d...


In [38]:
df_hash

,MESSAGE,TIME_OF_TRANSACTION,TRANSACTION_HASH,PUBLISHER_ADDRESS,SENT_ADDRESS
0,Success,2023-08-25 21:20:45,0x9ad586a86c548cff1de35f6e9daef09f2b44ff3a31bb...,0xa0a6ff7ee2d71d04c20195b8a919fafff513a9d4,0xb20016d23c77612334bd2885636980937da60b59


In [39]:
df_assets

,ASSET_CONTRACT,ASSET_ID,TIME_ASSET_CREATED,EPOCHS_NUMBER,EPOCH_LENGTH,TRAC_PRICE,EVENT,TRANSACTION_HASH,EVENT_CONTRACT_ADDRESS,BLOCK_HASH,BLOCK_NUMBER
0,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,561261,2023-08-25 21:20:18,5,7776000,0.02124,ServiceAgreementV1Created,0x9ad586a86c548cff1de35f6e9daef09f2b44ff3a31bb...,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F,0x86b72a67f9532e3f0ab7ebb691b26b2c3a6b31a1480d...,3120319
